In [82]:
import json
import pandas as pd
from datetime import datetime
from dateutil import relativedelta
import statistics as stat
import dateutil.relativedelta
import re

In [83]:
with open(r"flow-41611448_response (18).json", "r") as f:
        data1 = json.load(f)
#         crif=data1['sources']['values']['CRIF1']
#         data=data1['sources']['values']['CRIF1']['INDV-REPORT-FILE']['INDV-REPORTS'] 
        crif=data1['sources']['values']['crif_pull']
        data=data1['sources']['values']['crif_pull']['INDV-REPORT-FILE']['INDV-REPORTS'] 
# prospId =data1['sources']['values']['system_input']['IDVALUE']
# prospId =data1['sources']['values']['input']['Pan_No']
# prospId1=prospId
reqDate=datetime.now()

In [84]:
prospId='abc'

In [85]:
crifList = ['Age_Newest_TL', 'Age_Oldest_TL', 'Age_Oldest_TL_CC', 'amt_overdue_L24m', 'Avg_Age_Act', 'enq_L12m',
                'enq_L1m', 'enq_L1M_by_enq_L3M', 'enq_L3m', 'enq_L6m', 'First_Prod_CC_TL', 'high_credit_amount_by_bal',
                'Num_Closed_Secured_TL', 'Num_Closed_Unsecured_TL', 'num_deliq_12mts', 'Num_Open_CC_TL',
                'Num_Open_PL_TL', 'num_times_delinquent', 'num_TL_60DPD_L12M', 'num_tradelines',
                'pct_Satis_TL_to_Tot_TL', 'PROSPECTID', 'Secured_TL', 'time_since_recent_deliquency', 'Tot_Bal',
                'tot_deliq_TLs', 'Tot_Unsecured_Bal', 'Total_TL', 'Unsecured_TL', 'PL_TL', 'Tot_Non_CC_Active_TL',
                'WrittenoffAmount_L24M','WrittenoffAmount_L36M', 'WrittenoffAmount_L60M', 'Peak_EMI', 'Open_EMI', 'Debt_Burden_Ratio',
                'Peak_EMI_L36M', 'Active_HL_EMI_L12M', 'Credit_Vintage_or_Time_in_Bureau', 'Unsecured_EMI', 'DILI_EMI1',
                'DILI_EMI2', 'Credit_Limit_Income', 'Scienaptic_Income', 'Tot_Unsec_enq6m']

crifAccountList = ['Age_Newest_TL', 'Age_Oldest_TL', 'Age_Oldest_TL_CC', 'amt_overdue_L24m', 'Avg_Age_Act',
                       'First_Prod_CC_TL', 'high_credit_amount_by_bal', 'Num_Closed_Secured_TL',
                       'Num_Closed_Unsecured_TL', 'num_deliq_12mts', 'Num_Open_CC_TL', 'Num_Open_PL_TL',
                       'num_times_delinquent', 'num_TL_60DPD_L12M', 'num_tradelines', 'pct_Satis_TL_to_Tot_TL',
                       'PROSPECTID', 'Secured_TL', 'time_since_recent_deliquency', 'Tot_Bal', 'tot_deliq_TLs',
                       'Tot_Unsecured_Bal', 'Total_TL', 'Unsecured_TL', 'PL_TL', 'Tot_Non_CC_Active_TL',
                       'WrittenoffAmount_L24M','WrittenoffAmount_L36M', 'WrittenoffAmount_L60M', 'Peak_EMI', 'Open_EMI', 'Debt_Burden_Ratio',
                       'Peak_EMI_L36M', 'Active_HL_EMI_L12M', 'Credit_Vintage_or_Time_in_Bureau', 'Unsecured_EMI',
                       'DILI_EMI1', 'DILI_EMI2', 'Credit_Limit_Income', 'Scienaptic_Income']

crifEnquiryList = ['enq_L12m', 'enq_L1m', 'enq_L1M_by_enq_L3M', 'enq_L3m', 'enq_L6m', 'Tot_Unsec_enq6m']


In [86]:
stat_date = datetime.strptime('31/01/2000', '%d/%m/%Y')


In [87]:
def date_format(d):  # Change
        if d == '00-00-0000' or d == '' or d is None:
            d = '01-01-1900'
        # new_d = d[:2]+'/'+d[2:4]+'/'+d[4:]
        return datetime.strptime(d[:2] + '/' + d[2:4] + '/' + d[4:], '%d/%m/%Y')

In [88]:
def convert_app_date(d):
        app = None
        try:
            app = datetime.strptime(d[8:10] + '-' + d[4:7] + '-' + d[-4:], '%d-%b-%Y')
        except Exception as e:
            app = datetime.now()
            print(
                "Handling for Wrong Application Date format:::: Original {}  Converted {}".format(str(d), str(app)))
        return app

In [89]:
def get_account_report_list(crif):
    loan_list = crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INDV-RESPONSES']['INDV-RESPONSE-LIST']['INDV-RESPONSE']
    loan_list_new = []
    for i in loan_list:
        aa = i
        loan_list_new.append(aa)
    return loan_list_new


In [90]:
def date_format_inq(d):
    if d == '00-00-0000' or d == '' or d is None:
        d = '01-01-1900'
    return datetime.strptime(d[:2]+'/'+d[3:5]+'/'+d[6:], '%d/%m/%Y')

def date_format_payment_crif(d):
    if d == '00-00-0000' or d == '' or d is None:
        d = '01-01-1900'
    return datetime.strptime(d[6:]+'/'+d[3:5]+'/'+d[:2], '%Y/%m/%d')

def date_format_account_crif(d):
    if d == '20-06-2019 12:12:12' or d == '' or d is None:
        d = '01-01-1900'
    return datetime.strptime(d[6:]+'/'+d[3:5]+'/'+d[:2], '%Y/%m/%d')

def date_format_payment_crif_will_null(d):
    if d == '00-00-0000' or d == '' or d is None:
        return None
    return datetime.strptime(d[:2]+'/'+d[3:5]+'/'+d[6:], '%d/%m/%Y')

def date_format_payment(d):
    if d == '00000000' or d == '' or d is None:
        d = '01011900'
    return datetime.strptime(d[:-6]+'/'+d[-6:-4]+'/'+d[4:], '%d/%m/%Y')

def date_format_delq(d):
    if d == 'Mar:0000' or d == '' or d is None:
        d = 'Mar:2011'
    return datetime.strptime('01'+'/'+d[:-5]+'/'+d[4:], '%d/%b/%Y')


def date_format_payment_with_nulls(d):
    if d == '00000000' or d == '' or d is None:
        return None
    return datetime.strptime(d[:-6]+'/'+d[-6:-4]+'/'+d[4:], '%d/%m/%Y')


In [91]:
def get_tradeline(crif):
    loan_list = crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INDV-RESPONSES']['INDV-RESPONSE-LIST']['INDV-RESPONSE']
    loan_list_new = []
    if type(loan_list) == list:
        for i in loan_list:
            aa = i['LOAN-DETAIL']
            loan_list_new.append(aa)
    else:
        aa = crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['RESPONSES']['RESPONSE']['LOAN-DETAILS']
        loan_list_new.append(aa)
    return loan_list_new

In [92]:
def get_enquiry(crif):
    enquiry_list = crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INQUIRY-HISTORY']['HISTORY']
    enquiry_list_new = []
    if type(enquiry_list) == list:
        enquiry_list_new = crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INQUIRY-HISTORY']['HISTORY']
    else:
        aa = crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INQUIRY-HISTORY']['HISTORY']
        enquiry_list_new.append(aa)
    return enquiry_list_new

In [93]:
def identify_account_enquiry(crif):
    Account =get_tradeline(crif)
    accountPresent, enquiryPresent = True, True
    if type(Account) == list:
        if len(Account) == 0:
            accountPresent = False
    else:
        if Account is None or Account == "[]" or Account == "":
            accountPresent = False
    if type(crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INQUIRY-HISTORY']) == list:
        if len(crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INQUIRY-HISTORY']) == 0:
            enquiryPresent = False
    else:
        if crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INQUIRY-HISTORY'] is None or crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INQUIRY-HISTORY'] == "[]" or \
                crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INQUIRY-HISTORY'] == "":
            enquiryPresent = False
    return accountPresent, enquiryPresent

In [94]:
def date_format_delq(b):
    d = b[:8]
    if d == 'Mar:0000' or d == '' or d is None:
        d = 'Mar:2011'
    return datetime.strptime('01' + '/' + d[:3] + '/' + d[4:], '%d/%b/%Y')
def identify_account_or_enquiry(elem):
    if type(elem) == list:
        return True if len(elem) > 0 else False
    else:
        return True if elem != "" and elem is not None else False
def split_grid(s):
    return [s[i:i + 3] for i in range(0, len(s), 3)]
def check_occurence(splt, strcheck, mnth):
    if mnth is None:
        return splt.count(strcheck)
    else:
        return splt[mnth:].count(strcheck)

In [95]:
def get_split_paygrid_res(li, cond, strind, endind, func, eq):
        resp = None
        lis = [x for x in li[strind:endind] if x >= cond] if eq else [x for x in li[strind:endind] if x > cond]
        deflt = 0 if (func == 'sum') else None
        resp = sum(lis) if (len(lis) > 0 and func == 'sum') else max(lis) if (len(lis) > 0 and func == 'max') else len(
            lis) if func == 'count' else deflt
        return resp
def excl_none_operation(li, func):
    lis = [i for i in li if i is not None]
    if func == 'sum':
        return sum(lis) if len(lis) > 0 else None
    elif func == 'max':
        return max(lis) if len(lis) > 0 else None
    elif func == 'min':
        return min(lis) if len(lis) > 0 else None
    else:
        return stat.mean(lis) if len(lis) > 0 else None
def get_educ_cat(s):
    if s in ['SSC', '12TH', 'OTHERS']:
        return '1. SSC_12TH_OTHERS'
    elif s == 'UNDER GRADUATE':
        return '2. UNDER GRADUATE'
    elif s == 'GRADUATE':
        return '3. GRADUATE'
    elif s in ['POST-GRADUATE', 'PROFESSIONAL']:
        return '4. PG_PROFESSIONAL'
    else:
        return s
def last_first_prod_enq(s):
    s = str(s)
    if s == '10':
        return 'CC'
    elif s == '5' or s == '05':
        return 'PL'
    elif s == '2' or s == '02':
        return 'HL'
    elif s == '6' or s == '06':
        return 'ConsumerLoan'
    elif s == '1' or s == '01':
        return 'AL'
    else:
        return 'Others'
def replace_nones(d):
    resp = {}
    for k, v in d.items():
        resp[k] = -99999 if v is None else v
    return resp
def add_blank_crif(prospId, request, crif):
    resp = {}
    for k in crifList:
        resp[k] = -99999
    resp['PROSPECTID'] = prospId
    return resp
def add_blank_experian(prospId, request, experian):
    resp = {}
    for k in experianList:
        resp[k] = -99999
    resp['PROSPECTID'] = prospId
    resp['First_Prod_CC_or_not'] = '-99999'
    resp['current_date'] = '-99999'
    return resp
def get_required_columns(d):
    resp = {}
    for k, v in d.items():
        if k in crifList:
            resp[k] = v
    return resp
def add_nones(d):
    resp = {}
    for k in d:
        resp[k] = -99999
    return resp
def rev_sentence(sentence):
    # first split the string into words
    words = sentence.split(' ')
    # then reverse the split string list and join using space
    reverse_sentence = ''.join(reversed(words))
    # finally return the joined string
    return reverse_sentence
def get_surrogate_field(typ, surr):
    if typ not in surr:
        return False
    else:
        if surr[typ] is None:
            return False
        else:
            return True

def get_idList(crif):
    id_list = crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INDV-RESPONSES']['INDV-RESPONSE-LIST']['INDV-RESPONSE']
    id_list_new = []
    for i in id_list:
        aa = i['IDS']['ID']
        for i in aa:
            bb = i['TYPE']
            bb = {"TYPE": bb}
            id_list_new.append(bb)
    return id_list_new
def format_payment(d):
    months_remove = lambda x:"".join(re.sub('(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\:\d{4}\,','',x))
    months_remove_new = months_remove(d)
    remove_spec_char = lambda x:"".join(re.sub('\|', '',x))
    resp = remove_spec_char(months_remove_new)
    return resp
def format_payment_inst(d):
    months_remove = lambda x: "".join(re.sub(r'\D+', '', x))
    # months_remove = lambda x:"".join(re.sub('\\','',x))
    months_remove_new = months_remove(d)
    remove_spec_char1 = lambda x: "".join(x.replace("/", ''))
    resp = remove_spec_char1(months_remove_new)
    return resp
def format_payment_instnew(d):
    months_remove = lambda x: "".join(re.sub(r'\d+', '', x))
    months_remove_new = months_remove(d)
    remove_spec_char1 = lambda x: "".join(x.replace("/", ''))
    resp = remove_spec_char1(months_remove_new)
    remove_spec_char2 = lambda x: "".join(x.replace(",", ''))
    resp1 = remove_spec_char2(resp)
    return resp1
def rollup_crif_optional(prospId, crif, reqDate, optPresent):
    resp = {'PROSPECTID': prospId}
    if optPresent == 'INQUIRY-HISTORY':
        resp.update(add_nones(crifAccountList))
        resp.update(add_enquiry(prospId, crif, reqDate))
    else:
        resp.update(add_nones(crifEnquiryList))
        resp.update(add_account(prospId, crif, reqDate))
    return resp

In [96]:
def compare_and_handle_null(val1, val2):
        if val1 is None:
            return False
        else:
            return val1 >= val2

In [97]:
def add_enquiry(prospId, crif, reqDate):
        enquiry = crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['INQUIRY-HISTORY']['HISTORY']
        enquiry = [{**item, 'DATEREPORTED_new': date_format_inq(item['INQUIRY-DATE']), 'DISBURSEDDATE': reqDate,
                    'PURPOSE': (item['PURPOSE']),
                    'AMOUNT': ((item['AMOUNT']))} for item in enquiry]
        enquiry = [{**item, 'enquiries_L1m': item['DISBURSEDDATE'] - dateutil.relativedelta.relativedelta(months=1),
                    'enquiries_L3m': item['DISBURSEDDATE'] - dateutil.relativedelta.relativedelta(months=3),
                    'enquiries_L6m': item['DISBURSEDDATE'] - dateutil.relativedelta.relativedelta(months=6),
                    'enquiries_L12m': item['DISBURSEDDATE'] - dateutil.relativedelta.relativedelta(months=12),
                    'days_diff_disb_To_enqDate': (item['DISBURSEDDATE'] - item['DATEREPORTED_new']).days if item[
                                                                                                                'DISBURSEDDATE'] >
                                                                                                            item[
                                                                                                                'DATEREPORTED_new'] else 0}
                   for item in enquiry]

        # Credit Card Queries
        enq_purp10 = [item for item in enquiry if (item['PURPOSE'] == 'Auto Loan')]  # Total Credit card enquiries
        enq6m = [item for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L6m'] and item[
            'PURPOSE'] == 'Auto Loan')]  # CC queries in 6 months
        enq12m = [item for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L12m'] and item[
            'PURPOSE'] == 'Auto Loan')]  # CC queries in 12 months
        # Personal Loan Queries
        enq_loan = [item for item in enquiry if (item['PURPOSE'] == 'Personal Loan')]  # Total Personal Loan enquiries
        enq6pl = [item for item in enquiry if
                  (item['DATEREPORTED_new'] > item['enquiries_L6m'] and item['PURPOSE'] == 'Personal Loan')]
        enq12pl = [item for item in enquiry if
                   (item['DATEREPORTED_new'] > item['enquiries_L12m'] and item['PURPOSE'] == 'Personal Loan')]
        # Housing Loan Queroes
        enq_ProL = [item for item in enquiry if (item['PURPOSE'] == 'Property Loan')]  # Total House Loan enquiries
        enq6ProL = [item for item in enquiry if
                    (item['DATEREPORTED_new'] > item['enquiries_L6m'] and item['PURPOSE'] == 'Property Loan')]
        enq12ProL = [item for item in enquiry if
                     (item['DATEREPORTED_new'] > item['enquiries_L12m'] and item['PURPOSE'] == 'Property Loan')]
        # Durable Loan Queries
        enq_bl = [item for item in enquiry if
                  (item['PURPOSE'] == 'Business Loan General')]  # Total Personal Loan enquiries
        enq6bl = [item for item in enquiry if
                  (item['DATEREPORTED_new'] > item['enquiries_L6m'] and item['PURPOSE'] == 'Business Loan General')]
        enq12bl = [item for item in enquiry if
                   (item['DATEREPORTED_new'] > item['enquiries_L12m'] and item['PURPOSE'] == 'Business Loan General')]
        # Auto-Loan Queries
        enq_al = [item for item in enquiry if (item['PURPOSE'] == 'Business Loan General')]  # Total Auto Loan enquiries
        enq6al = [item for item in enquiry if
                  (item['DATEREPORTED_new'] > item['enquiries_L6m'] and item['PURPOSE'] == 'Auto Loan (Personal)')]
        enq12al = [item for item in enquiry if
                   (item['DATEREPORTED_new'] > item['enquiries_L12m'] and item['PURPOSE'] == 'Auto Loan (Personal)')]
        # Two-Wheeler Loan Queries
        enq_cvl = [item for item in enquiry if
                   (item['PURPOSE'] == 'Commercial Vehicle Loan')]  # Total Auto Loan enquiries
        # Used car loan Queries
        enq_ucl = [item for item in enquiry if (item['PURPOSE'] == 'Used Car Loan')]  # Total Used Car Loan enquiries

        tot_enq, tot_enq_p10, tot_enq6, tot_enq12 = len(enquiry), len(enq_purp10), len(enq6m), len(enq12m)
        tot_pl, tot_enq6pl, tot_enq12pl = len(enq_loan), len(enq6pl), len(enq12pl)
        tot_ProL, tot_enq6ProL, tot_enq12ProL = len(enq_ProL), len(enq6ProL), len(enq12ProL)
        tot_bl, tot_enq6bl, tot_enq12bl = len(enq_bl), len(enq6bl), len(enq12bl)
        tot_al, tot_enq6al, tot_enq12al = len(enq_al), len(enq6al), len(enq12al)
        tot_cvl, tot_ucl = len(enq_cvl), len(enq_ucl)  # NEW FIX

        enquiry = [{**item, 'recent_enq_inMnths': round(item['days_diff_disb_To_enqDate'] / 30)} for item in enquiry]

        enquiry_rollup = {'tot_enq': tot_enq, 'CC_enq': tot_enq_p10, 'CC_enq_L6m': tot_enq6, 'CC_enq_L12m': tot_enq12,
                          'PL_enq': tot_pl, 'PL_enq_L6m': tot_enq6pl, 'PL_enq_L12m': tot_enq12pl,
                          'PROL_enq': tot_ProL, 'PROL_enq_L6m': tot_enq6ProL, 'PROL_enq_L12m': tot_enq12ProL,
                          'BUSSL_enq': tot_bl, 'BUSSL_enq_L6m': tot_enq6bl, 'BUSSL_enq_L12m': tot_enq12bl,
                          'AL_enq': tot_al, 'AL_enq_L6m': tot_enq6al, 'AL_enq_L12m': tot_enq12al,
                          'CVL_enq': tot_cvl, 'UsedCarL_enq': tot_ucl}

        enquiry_rollup['tot_AL_enq'] = enquiry_rollup['AL_enq'] + enquiry_rollup['CVL_enq'] + enquiry_rollup[
            'UsedCarL_enq']
        enquiry_rollup['other_enq'] = enquiry_rollup['tot_enq'] - enquiry_rollup['CC_enq'] - enquiry_rollup['PL_enq'] - \
                                      enquiry_rollup['PROL_enq'] - enquiry_rollup['BUSSL_enq'] - enquiry_rollup[
                                          'tot_AL_enq']

        min_recency = min(d['days_diff_disb_To_enqDate'] for d in enquiry)
        max_recency = max(d['days_diff_disb_To_enqDate'] for d in enquiry)
        enquiry = [{**item, 'min_recency': min_recency, 'max_recency': max_recency} for item in enquiry]
        enq_dedup = sorted(enquiry, key=lambda i: i['days_diff_disb_To_enqDate'])
        enq_dedup1, enq_dedup2 = enq_dedup[0], enq_dedup[-1]
        enquiry_rollup['most_recent_enq_amt'] = enq_dedup1['AMOUNT']
        enquiry_rollup['last_prod_enq'] = enq_dedup1['PURPOSE']
        enquiry_rollup['time_since_recent_enq'] = enq_dedup1['min_recency']
        enquiry_rollup['first_prod_enq'] = enq_dedup2[
            'PURPOSE']  # CHANGE: Earlier was EnquiryAmount, it should be PURPOSE

        enq_L1m = ['1' for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L1m'])]
        enq_L3m = ['1' for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L3m'])]
        enq_L6m = ['1' for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L6m'])]
        enq_L12m = ['1' for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L12m'])]
        enquiry_rollup['enq_L1m'] = len(enq_L1m)
        enquiry_rollup['enq_L3m'] = len(enq_L3m)
        enquiry_rollup['enq_L6m'] = len(enq_L6m)
        enquiry_rollup['enq_L12m'] = len(enq_L12m)
        unsecured_list = ["Personal Loan", "Consumer Loan", "Credit Card", "Education Loan", "Loan to Professional",
                          "Credit Card", "Overdraft", "Fleet Card", "Telco Wireless", "Telco Broadband",
                          "Telco Landline",
                          "Kisan Credit Card", "Loan on Credit Card", "Prime Minister Jaan Dhan Yojana - Overdraft",
                          "Mudra Loans – Shishu / Kishor / Tarun", "Business Loan Unsecured"]
        Unsec_enq6m = [item for item in enquiry if
                       (item['DATEREPORTED_new'] > item['enquiries_L6m'] and item['PURPOSE'] in unsecured_list)]
        enquiry_rollup['Tot_Unsec_enq6m'] = len(Unsec_enq6m)
        enquiry_rollup['enq_L1M_by_enq_L3M'] = enquiry_rollup['enq_L1m'] / enquiry_rollup['enq_L3m'] if enquiry_rollup[
                                                                                                            'enq_L3m'] > 1 else None
        enquiry_rollup['PROSPECTID'] = prospId

        return enquiry_rollup

In [98]:
def open_grid(pat):
    s=list(pat)
    if(len(s) >0):
        s1= [-1 if x=='C' or x== 'N' or x== 'X' or x== 'Z' or x=='O' else int(x) for x in s]
    else:
        s1=['']
    
    return s1

In [99]:
def rollup_crif(prospId, crif, reqDate):
    #added if condition on 23/12/2021
    try:
        if(crif not in ['None',None,'',' ',[],{},[{}]]):
            enquiry, account = get_enquiry(crif), get_tradeline(crif)
            crif_tradlines = get_tradeline(crif)
            if(crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['SCORES']['SCORE'] is not None):
                    scoreList=crif['INDV-REPORT-FILE']['INDV-REPORTS']['INDV-REPORT']['SCORES']['SCORE']

            crif_tradlines = [d for d in crif_tradlines if 'ACCT-TYPE' in d]

            crif_tradlines = [{**item, 'DateClosed': None if 'CLOSED-DT' not in item or item[
                'CLOSED-DT'] == {} else date_format_payment_crif_will_null(item['CLOSED-DT']),
                               'DateOpenedOrDisbursed': None if 'DISBURSED-DT' not in item or item[
                                   'DISBURSED-DT'] == {} else date_format_payment_crif_will_null(
                                   item['DISBURSED-DT']),
                               'DateOfLastPayment': None if 'COMBINED-PAYMENT-HISTORY' not in item or item[
                                   'COMBINED-PAYMENT-HISTORY'] == {} else date_format_delq(
                                   item['COMBINED-PAYMENT-HISTORY']),
                               'CurrentBalance': 0 if item['CURRENT-BAL'] is None in item else item['CURRENT-BAL'],
                               'highCreditOrSanctionedAmount': 0 if 'DISBURSED-AMT' not in item or item[
                                   'DISBURSED-AMT'] == {} else int(str(item['DISBURSED-AMT']).replace(',', '')),
                               'InstallmentAmount1': 0 if 'INSTALLMENT-AMT' not in item or item[
                                   'INSTALLMENT-AMT'] == {} else item['INSTALLMENT-AMT'],
                               'InstallmentAmount2': None if 'INSTALLMENT-AMT' not in item or item[
                                   'INSTALLMENT-AMT'] == {} else 
                                   item['INSTALLMENT-AMT'],
                               'WrittenoffAmount': 0 if 'WRITE-OFF-AMT' not in item or item['WRITE-OFF-AMT'] == {} else int(
                                   str(item['WRITE-OFF-AMT']).replace(',', '')),
                               'Credit_Limit': 0 if 'CREDIT-LIMIT' not in item or item['CREDIT-LIMIT'] == {} else int(
                                   str(item['CREDIT-LIMIT']).replace(',', '')),
                               'DisbursedAmount': 0 if 'DISBURSED-AMT' not in item or item['DISBURSED-AMT'] == {} else int(
                                   str(item['DISBURSED-AMT']).replace(',', '')),
                               'PaymentHistoryEndDate': None if 'CLOSED-DT' not in item or item[
                                   'CLOSED-DT'] == {} else date_format_payment_crif_will_null(
                                   item['CLOSED-DT']),
                               'PaymentHistoryStartDate': None if 'DISBURSED-DT' not in item or item[
                                   'DISBURSED-DT'] == {} else date_format_payment_crif_will_null(
                                   item['DISBURSED-DT'])
                               } for item in crif_tradlines]

            crif_tradlines = [{**item, 'InstallmentAmount': item['InstallmentAmount1'] * 2 if item[
                                                                                                  'InstallmentAmount2'] == 'BiWeekly' else
            item['InstallmentAmount1']} for item in crif_tradlines]
            crif_tradlines = [{**item, 'Close_date_MissInd': 0 if item['DateClosed'] is None else 1} for item in
                              crif_tradlines]
            crif_tradlines = [{**item, 'Active_date_MissInd': 1 if item['DateClosed'] is None else 0} for item in
                              crif_tradlines]
            crif_tradlines = [{**item, 'Open_date_MissInd': 0 if item['DateOpenedOrDisbursed'] is None else 1} for item in
                              crif_tradlines]
            crif_tradlines = [{**item, 'Pmnt_date_MissInd': 0 if item['DateOfLastPayment'] is None else 1} for item in
                              crif_tradlines]
            crif_tradlines = [{**item, 'CCInd': 1 if item['ACCT-TYPE'] == 'Credit Card' else 0} for item in crif_tradlines]
            crif_tradlines = [{**item, 'MissInd_Pmnt_date': 1 if item['CurrentBalance'] > 0 is None and item[
                'DateOfLastPayment'] is None else 0} for item in crif_tradlines]

            crif_tradlines = [{**item, 'DATECLOSED_New': reqDate if (item['DateClosed']) is None else (item['DateClosed']),
                               'DATEOFLASTPAYMENT_New': item['DateOfLastPayment'],
                               'DATEOPENEDORDISBURSED_New': item['DateOpenedOrDisbursed'],
                               'PAYMENTHISTORYENDDATE_New': item['PaymentHistoryEndDate'],
                               'PAYMENTHISTORYSTARTDATE_New': item['PaymentHistoryStartDate']} for item in crif_tradlines]

            nones = [i for i in crif_tradlines if i['DATEOPENEDORDISBURSED_New'] is None]
            nonnoes = sorted([i for i in crif_tradlines if i['DATEOPENEDORDISBURSED_New'] is not None],
                             key=lambda i: i['DATEOPENEDORDISBURSED_New'])
            crif_tradlines = nonnoes + nones
            disb = {'PROSPECTID': prospId, 'DISBDATE': reqDate}
            disb['L3M'] = disb['DISBDATE'] - dateutil.relativedelta.relativedelta(months=3)
            disb['L6M'] = disb['DISBDATE'] - dateutil.relativedelta.relativedelta(months=6)
            disb['L12M'] = disb['DISBDATE'] - dateutil.relativedelta.relativedelta(months=12)
            disb['L24M'] = disb['DISBDATE'] - dateutil.relativedelta.relativedelta(months=24)
            disb['L36M'] = disb['DISBDATE'] - dateutil.relativedelta.relativedelta(months=36)
            disb['L60M'] = disb['DISBDATE'] - dateutil.relativedelta.relativedelta(months=60)
            crif_tradlines = [
                {**item, 'DISBDATE': disb['DISBDATE'],'L3M': disb['L3M'], 'L6M': disb['L6M'], 'L12M': disb['L12M'], 'L24M': disb['L24M'],
                 'L36M': disb['L36M'], 'L60M': disb['L60M']} for item in crif_tradlines]
            del disb

            trades = {'PROSPECTID': prospId, 'Total_TL': sum(d['Open_date_MissInd'] for d in crif_tradlines),
                      'Tot_Active_TL': sum(
                          d['Open_date_MissInd'] == 1 and (d['Close_date_MissInd'] == 0) for d in crif_tradlines),
                      'Tot_Closed_TL': sum(
                          d['Open_date_MissInd'] == 1 and (d['Close_date_MissInd'] == 1) for d in crif_tradlines),
                      'Tot_Non_CC_Active_TL': sum(
                          d['Open_date_MissInd'] == 1 and (d['Close_date_MissInd'] == 0) and (d['CCInd'] == 0) for d in
                          crif_tradlines)}
            trades['Neither_Active_Nor_Closed'] = trades['Total_TL'] - trades['Tot_Closed_TL'] - trades['Tot_Active_TL']
            crif_tradlines = [{**item, 'closed_opened_diff_mts': None if (item['DATECLOSED_New']) is None else (
                        (item['DATECLOSED_New'] - item['DATEOPENEDORDISBURSED_New']).days / 30)} for item in crif_tradlines]
            trades['Avg_Age_Act'] = excl_none_operation([d['closed_opened_diff_mts'] for d in crif_tradlines],
                                                                     'mean')

            L6M_accounts = [item for item in crif_tradlines if
                            compare_and_handle_null(item['DATEOPENEDORDISBURSED_New'], item['L6M'])]

            L3M_accounts = [item for item in crif_tradlines if
                                compare_and_handle_null(item['DATEOPENEDORDISBURSED_New'], item['L3M'])]

            L6M_closed = [item for item in crif_tradlines if
                          compare_and_handle_null(item['DATECLOSED_New'], item['L6M'])]
            L12M_accounts = [item for item in crif_tradlines if
                             compare_and_handle_null(item['DATEOPENEDORDISBURSED_New'], item['L12M'])]
            L12M_closed = [item for item in crif_tradlines if
                           compare_and_handle_null(item['DATECLOSED_New'], item['L12M'])]

            L6M_payment = [item for item in crif_tradlines if
                           compare_and_handle_null(item['DATEOFLASTPAYMENT_New'], item['L6M'])]
            L12M_payment = [item for item in crif_tradlines if
                            compare_and_handle_null(item['DATEOFLASTPAYMENT_New'], item['L12M'])]

            x1 = [item for item in crif_tradlines if item['Open_date_MissInd'] == 1]
            trades['Total_TL_opened_L6M'] = sum(d['Open_date_MissInd'] for d in x1 and L6M_accounts)
            trades['Total_TL_opened_L3M']= sum(d['Open_date_MissInd'] for d in x1 and L3M_accounts)

            trades['Total_TL_opened_L12M'] = sum(d['Open_date_MissInd'] for d in x1 and L12M_accounts)
            x2 = [item for item in crif_tradlines if (item['Open_date_MissInd'] == 1 and item['Close_date_MissInd'] == 1)]
            trades['Tot_TL_closed_L6M'] = sum(d['Open_date_MissInd'] for d in x2 and L6M_closed)
            trades['Tot_TL_closed_L12M'] = sum(d['Open_date_MissInd'] for d in x2 and L12M_closed)
            x3 = [item for item in crif_tradlines if (item['Open_date_MissInd'] == 1 and item['Close_date_MissInd'] == 0)]
            trades['Tot_Active_TL_L6M'] = sum(d['Open_date_MissInd'] for d in x3 and L6M_accounts)
            trades['Tot_Active_TL_L12M'] = sum(d['Open_date_MissInd'] for d in x3 and L12M_accounts)
            # Total missed payment made accross TL
            x4 = [item for item in crif_tradlines if (item['Open_date_MissInd'] == 1 and item['MissInd_Pmnt_date'] == 1)]
            trades['Tot_Missed_Pmnt'] = sum(d['MissInd_Pmnt_date'] for d in x4 and x4)
            x5 = [item for item in crif_tradlines if (item['Open_date_MissInd'] == 1 and item['Pmnt_date_MissInd'] == 1)]
            trades['Tot_Num_Pmnt'] = sum(d['Pmnt_date_MissInd'] for d in x5 and x5)
            # Total number payment made accross TL in last 6 months
            trades['Tot_Num_Pmnt_L6M'] = sum(d['Pmnt_date_MissInd'] for d in x1 and L6M_payment)
            trades['Tot_Num_Pmnt_L12M'] = sum(d['Pmnt_date_MissInd'] for d in x1 and L12M_payment)

            crif_tradlines = [{**item, 'Total_TL': trades['Total_TL'], 'Tot_Closed_TL': trades['Tot_Closed_TL'],
                               'Tot_Active_TL': trades['Tot_Active_TL'],
                               'Total_TL_opened_L3M': trades['Total_TL_opened_L3M'],
                               'Total_TL_opened_L6M': trades['Total_TL_opened_L6M'],
                               'Tot_TL_closed_L6M': trades['Tot_TL_closed_L6M'],
                               'Total_TL_opened_L12M': trades['Total_TL_opened_L12M'],
                               'Tot_TL_closed_L12M': trades['Tot_TL_closed_L12M'],
                               'Tot_Missed_Pmnt': trades['Tot_Missed_Pmnt'],
                               'HIGHCREDITORSANCTIONEDAMOUNT': item[
                                   'highCreditOrSanctionedAmount'] if 'highCreditOrSanctionedAmount' in item else None,
                               'ACCOUNTTYPE': item['ACCT-TYPE']} for item in crif_tradlines]

            # Types of Trades
            secured_list = ["Auto Loan (Personal)", "Housing Loan", "Property Loan", "Loan Against Shares / Securities",
                            "Gold Loan", "Leasing", "Two-Wheeler Loan", "Non-Funded Credit Facility",
                            "Loan Against Bank Deposits", "Commercial Vehicle Loan", "Secured Credit Card", "Used Car Loan",
                            "Construction Equipment Loan", "Used Tractor Loan", "Corporate Credit Card",
                            "Microfinance Business Loan", "Microfinance Personal Loan", "Microfinance Housing Loan",
                            "Microfinance Others", "Business Loan - Secured", "Business Loan General",
                            "Business Loan Priority Sector Small Business", "Business Loan Priority Sector Agriculture",
                            "Business Loan Priority Sector Others", "Business Non-Funded Credit Facility General",
                            "Business Non-Funded Credit Facility-Priority Sector- Small Business",
                            "Business Non-Funded Credit Facility-Priority Sector-Agriculture",
                            "Business Non-Funded Credit Facility-Priority Sector-Others",
                            "Business Loan Against Bank Deposits"]
            unsecured_list = ["Personal Loan", "Consumer Loan", "Credit Card", "Education Loan", "Loan to Professional",
                              "Credit Card", "Overdraft", "Fleet Card", "Telco Wireless", "Telco Broadband",
                              "Telco Landline", "Kisan Credit Card", "Loan on Credit Card",
                              "Prime Minister Jaan Dhan Yojana - Overdraft", "Mudra Loans – Shishu / Kishor / Tarun",
                              "Business Loan Unsecured"]

            crif_tradlines = [{**item, 'Gold_TL': 1 if item['ACCOUNTTYPE'] == "Gold Loan" else 0,
                               'Consumer_TL': 1 if item['ACCOUNTTYPE'] == "Consumer Loan" else 0,
                               'CC_TL': 1 if item['ACCOUNTTYPE'] == "Credit Card" else 0,
                               'Auto_TL': 1 if item['ACCOUNTTYPE'] in ["Auto Loan (Personal)", "Two-Wheeler Loan",
                                                                       "Used Car Loan"] else 0,
                               'Home_TL': 1 if item['ACCOUNTTYPE'] == "Housing Loan" else 0,
                               'PL_TL': 1 if item['ACCOUNTTYPE'] == "Personal Loan" else 0,
                               'Other_TL': 0 if item['ACCOUNTTYPE'] not in ["JLG Group", "JLG Individual",
                                                                            "Individual"] else 1,
                               'Secured_TL': 1 if item['ACCOUNTTYPE'] in secured_list else 0,
                               'Unsecured_TL': 1 if item['ACCOUNTTYPE'] in unsecured_list else 0
                               } for item in crif_tradlines]

            trades['Gold_TL'] = sum(d['Gold_TL'] for d in crif_tradlines)
            trades['Consumer_TL'] = sum(d['Consumer_TL'] for d in crif_tradlines)
            trades['CC_TL'] = sum(d['CC_TL'] for d in crif_tradlines)
            trades['Auto_TL'] = sum(d['Auto_TL'] for d in crif_tradlines)
            trades['Home_TL'] = sum(d['Home_TL'] for d in crif_tradlines)
            trades['PL_TL'] = sum(d['PL_TL'] for d in crif_tradlines)
            trades['Other_TL'] = sum(d['Other_TL'] for d in crif_tradlines)
            trades['Secured_TL'] = sum(d['Secured_TL'] for d in crif_tradlines)
            trades['Unsecured_TL'] = sum(d['Unsecured_TL'] for d in crif_tradlines)

            x3 = [item for item in crif_tradlines if (item['Open_date_MissInd'] == 1 and item['Close_date_MissInd'] == 0)]
            x3_data = [{**item, 'Num_Open_Gold_TL': 1 if item['ACCOUNTTYPE'] == "Gold Loan" else 0,
                        'Num_Open_Consumer_TL': 1 if item['ACCOUNTTYPE'] == "Consumer Loan" else 0,
                        'Num_Open_CC_TL': 1 if item['ACCOUNTTYPE'] == "Credit Card" else 0,
                        'Num_Open_Auto_TL': 1 if item['ACCOUNTTYPE'] in ["Auto Loan (Personal)", "Two-Wheeler Loan",
                                                                         "Used Car Loan"] else 0,
                        'Num_Open_Home_TL': 1 if item['ACCOUNTTYPE'] == "Housing Loan" else 0,
                        'Num_Open_PL_TL': 1 if item['ACCOUNTTYPE'] == "Personal Loan" else 0,
                        'Num_Open_Other_TL': 1 if item['ACCOUNTTYPE'] not in ["JLG Group", "JLG Individual",
                                                                              "Individual"] else 0,
                        'Num_Open_Secured_TL': 1 if item['ACCOUNTTYPE'] in secured_list else 0,
                        'Num_Open_Unsecured_TL': 1 if item['ACCOUNTTYPE'] in unsecured_list else 0
                        } for item in x3]

            trades['Num_Open_Gold_TL'] = sum(d['Num_Open_Gold_TL'] for d in x3_data)
            trades['Num_Open_Consumer_TL'] = sum(d['Num_Open_Consumer_TL'] for d in x3_data)
            trades['Num_Open_CC_TL'] = sum(d['Num_Open_CC_TL'] for d in x3_data)
            trades['Num_Open_Auto_TL'] = sum(d['Num_Open_Auto_TL'] for d in x3_data)
            trades['Num_Open_Home_TL'] = sum(d['Num_Open_Home_TL'] for d in x3_data)
            trades['Num_Open_PL_TL'] = sum(d['Num_Open_PL_TL'] for d in x3_data)
            trades['Num_Open_Other_TL'] = sum(d['Num_Open_Other_TL'] for d in x3_data)
            trades['Num_Open_Secured_TL'] = sum(d['Num_Open_Secured_TL'] for d in x3_data)
            trades['Num_Open_Unsecured_TL'] = sum(d['Num_Open_Unsecured_TL'] for d in x3_data)

            x2 = [item for item in crif_tradlines if (item['Open_date_MissInd'] == 1 and item['Close_date_MissInd'] == 1)]
            x2_data = [{**item, 'Num_Closed_Gold_TL': 1 if item['ACCOUNTTYPE'] == "Gold Loan" else 0,
                        'Num_Closed_Consumer_TL': 1 if item['ACCOUNTTYPE'] == "Consumer Loan" else 0,
                        'Num_Closed_CC_TL': 1 if item['ACCOUNTTYPE'] == "Credit Card" else 0,
                        'Num_Closed_Auto_TL': 1 if item['ACCOUNTTYPE'] in ["Auto Loan (Personal)", "Two-Wheeler Loan",
                                                                           "Used Car Loan"] else 0,
                        'Num_Closed_Home_TL': 1 if item['ACCOUNTTYPE'] == "Housing Loan" else 0,
                        'Num_Closed_PL_TL': 1 if item['ACCOUNTTYPE'] == "Personal Loan" else 0,
                        'Num_Closed_Other_TL': 1 if item['ACCOUNTTYPE'] not in ["JLG Group", "JLG Individual",
                                                                                "Individual"] else 0,
                        'Num_Closed_Secured_TL': 1 if item['ACCOUNTTYPE'] in secured_list else 0,
                        'Num_Closed_Unsecured_TL': 1 if item['ACCOUNTTYPE'] in unsecured_list else 0
                        } for item in x2]

            trades['Num_Closed_Gold_TL'] = sum(d['Num_Closed_Gold_TL'] for d in x2_data)
            trades['Num_Closed_Consumer_TL'] = sum(d['Num_Closed_Consumer_TL'] for d in x2_data)
            trades['Num_Closed_CC_TL'] = sum(d['Num_Closed_CC_TL'] for d in x2_data)
            trades['Num_Closed_Auto_TL'] = sum(d['Num_Closed_Auto_TL'] for d in x2_data)
            trades['Num_Closed_Home_TL'] = sum(d['Num_Closed_Home_TL'] for d in x2_data)
            trades['Num_Closed_PL_TL'] = sum(d['Num_Closed_PL_TL'] for d in x2_data)
            trades['Num_Closed_Other_TL'] = sum(d['Num_Closed_Other_TL'] for d in x2_data)
            trades['Num_Closed_Secured_TL'] = sum(d['Num_Closed_Secured_TL'] for d in x2_data)
            trades['Num_Closed_Unsecured_TL'] = sum(d['Num_Closed_Unsecured_TL'] for d in x2_data)

            trades['Num_PL_TL'] = sum([trades['Num_Open_PL_TL'], trades['Num_Closed_PL_TL']])

            ######
            x1date = datetime.strptime('11/6/1920', '%d/%m/%Y')
            #    x1, x = [item for item in cibil_trdlines if (item['Open_date_MissInd'] == 1 and compare_and_handle_null(item['DATEOPENEDORDISBURSED_New'], x1date))], {}
            x1, x = [item for item in crif_tradlines if (
                        item['Open_date_MissInd'] == 1 and compare_and_handle_null(
                    item['DATEOPENEDORDISBURSED_New'], x1date))], {}
            dateOpenedDis = [item['DATEOPENEDORDISBURSED_New'] for item in x1]

            x['Min_Opend_Date'] = min(dateOpenedDis) if len(dateOpenedDis) > 0 else x1date
            x['Max_Opend_Date'] = max(dateOpenedDis) if len(dateOpenedDis) > 0 else x1date
            crif_tradlines = [{**item, 'Min_Opend_Date': x['Min_Opend_Date'], 'Max_Opend_Date': x['Max_Opend_Date'],
                               'Age_Oldest_TL': (item['DISBDATE'] - x['Min_Opend_Date']).days / 30,
                               'Age_Newest_TL': (item['DISBDATE'] - x['Max_Opend_Date']).days / 30,
                               'Age_Oldest_TL_CC': (item['DISBDATE'] - x['Max_Opend_Date']).days / 30 if item[
                                                                                                             'ACCOUNTTYPE'] == 'Credit Card' else 0,
                               'Time_in_Bureau': (reqDate - item['DATEOPENEDORDISBURSED_New']).days / 30} for item in
                              crif_tradlines]
            # Dec 23 - Change - Start
            nones = [i for i in crif_tradlines if i['DATEOPENEDORDISBURSED_New'] is None]
            nonnoes = sorted([i for i in crif_tradlines if i['DATEOPENEDORDISBURSED_New'] is not None],
                             key=lambda i: i['DATEOPENEDORDISBURSED_New'])
            tl_age = nonnoes + nones
            tl_age_new = [item for item in tl_age if (item['ACCOUNTTYPE'] == 'Credit Card')]
            # Dec 23 - Change end
            # tl_age = sorted(cibil_trdlines, key=lambda i: i['DATEOPENEDORDISBURSED_New'])

            if len(tl_age) > 0:
                tl_age = tl_age[0]
                trades['Age_Oldest_TL'] = tl_age['Age_Oldest_TL']
                trades['Age_Newest_TL'] = tl_age['Age_Newest_TL']
            else:
                trades['Age_Oldest_TL'] = None
                trades['Age_Newest_TL'] = None

            if len(tl_age_new) > 0:
                tl_age_new = tl_age_new[0]
                trades['Age_Oldest_TL_CC'] = tl_age_new['Age_Oldest_TL_CC']
            else:
                trades['Age_Oldest_TL_CC'] = None

            #############  Current Balance  Amount Variable
            x1 = [item for item in crif_tradlines if (item['Open_date_MissInd'] == 1 and item['Close_date_MissInd'] == 0)]
            trades['Tot_Curr_bal_Accross_allOpen_TL'] = sum(int(d['CurrentBalance']) for d in x1)
            # added 26/10/2021 -start
            trades['debt_curr_bal'] = sum(int(d['CurrentBalance']) if d['ACCT-TYPE'] != "Auto Loan (Personal)" or d['ACCT-TYPE'] != "Two-Wheeler Loan" or d['ACCT-TYPE'] != "Used Car Loan" or d['ACCT-TYPE'] != "Commercial Vehicle Loan" or d['ACCT-TYPE'] != "Used Tractor Loan" or d['ACCT-TYPE'] != "Housing Loan" or d['ACCT-TYPE'] != "Property Loan" else 0 for d in x1)
            # --end 26/10/2021
            del x1
            L12M_List = [item for item in crif_tradlines if (item['DISBDATE'] > item['L12M'])]
            L24M_List = [item for item in crif_tradlines if (item['DISBDATE'] > item['L24M'])]
            L36M_List = [item for item in crif_tradlines if (item['DISBDATE'] > item['L36M'])]
            L60M_List = [item for item in crif_tradlines if (item['DISBDATE'] > item['L60M'])]
            L12M_List = [item for item in crif_tradlines if (
                        item['DISBDATE'] > item['L12M'] and item['Close_date_MissInd'] == 1 and item[
                    'ACCT-TYPE'] == 'Housing Loan')]
            CC_List = [item for item in crif_tradlines if (item['ACCT-TYPE'] == 'Credit Card')]

            # added 25/11/2021
            trades['WrittenoffAmount_L12M'] = sum(int(d['WrittenoffAmount']) for d in L12M_List)
            trades['WrittenoffAmount_L24M'] = sum(int(d['WrittenoffAmount']) for d in L24M_List)
            trades['WrittenoffAmount_L60M'] = sum(int(d['WrittenoffAmount']) for d in L60M_List)
            trades['WrittenoffAmount_L36M'] = sum(int(d['WrittenoffAmount']) for d in L36M_List)

            trades['Credit_Limit_Income'] = sum(int(d['Credit_Limit']) for d in CC_List)
            trades['Scienaptic_Income'] = trades['Credit_Limit_Income'] / 4 if trades['Credit_Limit_Income'] > 100000 else \
            trades['Credit_Limit_Income'] / 2

            x_1 = [item for item in crif_tradlines if (item['Open_date_MissInd'] == 1 and item['Close_date_MissInd'] == 0)]

            trades['Peak_EMI'] = sum(int(d['InstallmentAmount']) for d in crif_tradlines)
            trades['Open_EMI'] = sum(int(d['InstallmentAmount']) for d in x_1)
            del x_1

            invalid_values = [0, None]
            trades['Debt_Burden_Ratio'] = (trades['Open_EMI'] / trades['Peak_EMI']) if trades[
                                                                                           'Peak_EMI'] not in invalid_values and \
                                                                                       trades['Open_EMI'] is not None else 0
            trades['Peak_EMI_L36M'] = sum(int(d['InstallmentAmount']) for d in L36M_List)
            trades['Active_HL_EMI_L12M'] = sum(int(d['InstallmentAmount']) for d in L12M_List)
            trades['Credit_Vintage_or_Time_in_Bureau'] = max(int(d['Time_in_Bureau']) for d in crif_tradlines)

            USL_BAL = [item for item in crif_tradlines if item['Open_date_MissInd'] == 1 and item['Close_date_MissInd'] == 0 and item['Unsecured_TL'] == 1]
            SL_BAL = [item for item in crif_tradlines if item['Open_date_MissInd'] == 1 and item['Close_date_MissInd'] == 0 and item['Secured_TL'] == 1]

            trades['Tot_Secured_Bal'] = excl_none_operation([d['CurrentBalance'] for d in SL_BAL], 'sum')
            trades['Tot_Unsecured_Bal'] = excl_none_operation([d['CurrentBalance'] for d in USL_BAL], 'sum')
            trades['tot_UnsecSanc_amt'] = excl_none_operation([d['HIGHCREDITORSANCTIONEDAMOUNT'] for d in USL_BAL], 'sum')

            x1 = [item for item in crif_tradlines if (item['Open_date_MissInd'] == 1 and item['Close_date_MissInd'] == 0)]
            x1_data = [{**item, 'Secured_TL_F': 1 if item['ACCOUNTTYPE'] in secured_list else 0,
                        'Unsecured_TL_F': 1 if item['ACCOUNTTYPE'] not in unsecured_list else 0} for item in x1]
            trades['Tot_Bal'] = sum(int(d['CurrentBalance']) for d in x1_data and x1_data)

            x2_data = [{**item, 'Secured_TL_F': 1 if item['ACCOUNTTYPE'] in secured_list else 0,
                        'Unsecured_TL_F': 1 if item['ACCOUNTTYPE'] not in unsecured_list else 0} for item in crif_tradlines]
            USL_Bal1 = [item for item in crif_tradlines if (item['Unsecured_TL_F'] == 1 for item in x2_data)]
            trades['Unsecured_EMI'] = sum(int(d['InstallmentAmount']) for d in x2_data and USL_Bal1)

            trades['DILI_EMI1'] = (trades['Peak_EMI_L36M'] / trades['Open_EMI']) if trades['Open_EMI'] not in invalid_values and trades['Peak_EMI_L36M'] is not None else 0
            trades['DILI_EMI2'] = (0.2 * trades['Active_HL_EMI_L12M'] / trades['Unsecured_EMI']) if trades['Unsecured_EMI'] not in invalid_values and trades['Active_HL_EMI_L12M'] is not None else 0
            del x1, x1_data, x2_data, USL_Bal1, USL_BAL, SL_BAL

            x1 = [item for item in crif_tradlines if (item['Close_date_MissInd'] == 0)]
            # Sanctioned  Amount Variable
            trades['Min_Sactioned_amt'] = excl_none_operation([d['HIGHCREDITORSANCTIONEDAMOUNT'] for d in x1],'min')
            trades['Max_Sactioned_amt'] = excl_none_operation([d['HIGHCREDITORSANCTIONEDAMOUNT'] for d in x1],'max')
            trades['Tot_Sactioned_amt'] = excl_none_operation([d['HIGHCREDITORSANCTIONEDAMOUNT'] for d in x1],'sum')

            # Balance Related Variable
            GL_Limit = [item for item in crif_tradlines if item['ACCOUNTTYPE'] == "Gold Loan"]
            CC_Limit = [item for item in crif_tradlines if item['ACCOUNTTYPE'] == "Credit Card"]
            PL_Limit = [item for item in crif_tradlines if item['ACCOUNTTYPE'] == "Personal Loan"]
            HL_Limit = [item for item in crif_tradlines if item['ACCOUNTTYPE'] == "Housing Loan"]
            AL_Limit = [item for item in crif_tradlines if item['ACCOUNTTYPE'] == ["Auto Loan (Personal)", "Two-Wheeler Loan", "Used Car Loan"]]
            Other_Limit = [item for item in crif_tradlines if item['ACCOUNTTYPE'] != ["Auto Loan (Personal)", "Two-Wheeler Loan", "Used Car Loan", "Gold Loan","Credit Card", "Personal Loan", "Housing Loan"]]

            trades['Tot_CC_Bal'] = excl_none_operation([int(d['CurrentBalance']) for d in CC_Limit], 'sum')
            trades['Tot_PL_Bal'] = excl_none_operation([int(d['CurrentBalance']) for d in PL_Limit], 'sum')
            trades['Tot_HL_Bal'] = excl_none_operation([int(d['CurrentBalance']) for d in HL_Limit], 'sum')
            trades['Tot_AL_Bal'] = excl_none_operation([int(d['CurrentBalance']) for d in AL_Limit], 'sum')
            trades['Tot_GL_Bal'] = excl_none_operation([int(d['CurrentBalance']) for d in GL_Limit], 'sum')
            trades['Tot_Oth_Bal'] = excl_none_operation([int(d['CurrentBalance']) for d in Other_Limit], 'sum')

            # Total Revolving Balance
            trades['Tot_Revolving_Bal'] = trades['Tot_CC_Bal']

            # Total Closed Balance
            # trades['Tot_Closed_TL'] = sum(1 for item in crif_tradlines if 'ACCOUNT-STATUS' in item and item['ACCOUNT-STATUS'] == 'Closed')
            # trades['Total_TL'] = len(crif_tradlines)
            # print("ASHISH", [float(item['DISBURSED-AMT']) for item in crif_tradlines],
            #       [float(item['CURRENT-BAL']) for item in crif_tradlines])

            try:
                trades['Total_cur_blnc_to_Original_amount'] = excl_none_operation([int(d['highCreditOrSanctionedAmount']) for d in crif_tradlines], 'sum') / excl_none_operation([int(d['CurrentBalance']) for d in crif_tradlines], 'sum')
            except:
                trades['Total_cur_blnc_to_Original_amount'] = -99999
            print(CC_Limit)
            try:
                trades['util'] = excl_none_operation([int(d['highCreditOrSanctionedAmount']) for d in CC_Limit], 'sum') / excl_none_operation([int(d['CurrentBalance']) for d in CC_Limit], 'sum')
            except:
                trades['util'] = -99999

            # Limit related Variable
            trades['Tot_CC_Limit'] = excl_none_operation([int(d['DisbursedAmount']) for d in CC_Limit], 'sum')
            trades['Tot_PL_Limit'] = excl_none_operation([int(d['DisbursedAmount']) for d in PL_Limit], 'sum')
            trades['Tot_HL_Limit'] = excl_none_operation([int(d['DisbursedAmount']) for d in HL_Limit], 'sum')
            trades['Tot_AL_Limit'] = excl_none_operation([int(d['DisbursedAmount']) for d in AL_Limit], 'sum')
            trades['Tot_GL_Limit'] = excl_none_operation([int(d['DisbursedAmount']) for d in GL_Limit], 'sum')
            trades['Tot_Oth_Limit'] = excl_none_operation([int(d['DisbursedAmount']) for d in Other_Limit],
                                                                       'sum')

            del GL_Limit
            del CC_Limit
            del PL_Limit
            del HL_Limit
            del AL_Limit

            x1 = [item for item in crif_tradlines if (item['Open_date_MissInd'] == 1 and item['Close_date_MissInd'] == 1)]
            # Time since recent payment

            Last_Payment = excl_none_operation(
                [d['DATEOFLASTPAYMENT_New'] for d in x1 if d['DATEOFLASTPAYMENT_New'] is not None], 'max')

            crif_tradlines = [{**item, 'Last_Payment': Last_Payment} for item in crif_tradlines]
            crif_tradlines = [{**item, 'Time_Since_Payment': -99999 if item['Last_Payment'] is None else (
                        item['DISBDATE'] - item['Last_Payment']).days} for item in crif_tradlines]
            crif_tradlines = [{**item,
                               'Time_Since_Last_Payment_inMnths': -99999 if item['Time_Since_Payment'] is None else item[
                                                                                                                        'Time_Since_Payment'] / 30}
                              for item in crif_tradlines]

            nones = [i for i in crif_tradlines if i['DATEOPENEDORDISBURSED_New'] is None]
            nonnoes = sorted([i for i in crif_tradlines if i['DATEOPENEDORDISBURSED_New'] is not None],
                             key=lambda i: i['DATEOPENEDORDISBURSED_New'], reverse=True)
            TS_Last_Payment = nonnoes + nones

            if len(TS_Last_Payment) > 0:
                TS_Last_Payment = TS_Last_Payment[0]
                trades['Time_Since_Payment'] = TS_Last_Payment['Time_Since_Payment']
                trades['Time_Since_Last_Payment_inMnths'] = round(trades['Time_Since_Payment'] / 30)
            else:
                trades['Time_Since_Payment'] = None
                trades['Time_Since_Last_Payment_inMnths'] = None

            enquiry = [
                {**item, 'DATEREPORTED_new': date_format_inq(item['INQUIRY-DATE']), 'DISBURSEDDATE': reqDate,
                 'PURPOSE': (item['PURPOSE']),
                 'AMOUNT': ((item['AMOUNT']))} for item in enquiry]
            enquiry = [{**item, 'enquiries_L1m': item['DISBURSEDDATE'] - dateutil.relativedelta.relativedelta(months=1),
                        'enquiries_L3m': item['DISBURSEDDATE'] - dateutil.relativedelta.relativedelta(months=3),
                        'enquiries_L6m': item['DISBURSEDDATE'] - dateutil.relativedelta.relativedelta(months=6),
                        'enquiries_L12m': item['DISBURSEDDATE'] - dateutil.relativedelta.relativedelta(months=12),
                        'days_diff_disb_To_enqDate': (item['DISBURSEDDATE'] - item['DATEREPORTED_new']).days if item[
                                                                                                                    'DISBURSEDDATE'] >
                                                                                                                item[
                                                                                                                    'DATEREPORTED_new'] else 0}
                       for item in enquiry]

            # Credit Card Queries
            enq_purp10 = [item for item in enquiry if (item['PURPOSE'] == 'Auto Loan')]  # Total Credit card enquiries
            enq6m = [item for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L6m'] and item[
                'PURPOSE'] == 'Auto Loan')]  # CC queries in 6 months
            enq12m = [item for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L12m'] and item[
                'PURPOSE'] == 'Auto Loan')]  # CC queries in 12 months
            # Personal Loan Queries
            enq_loan = [item for item in enquiry if (item['PURPOSE'] == 'Personal Loan')]  # Total Personal Loan enquiries
            enq6pl = [item for item in enquiry if
                      (item['DATEREPORTED_new'] > item['enquiries_L6m'] and item['PURPOSE'] == 'Personal Loan')]
            enq12pl = [item for item in enquiry if
                       (item['DATEREPORTED_new'] > item['enquiries_L12m'] and item['PURPOSE'] == 'Personal Loan')]
            # Housing Loan Queroes
            enq_ProL = [item for item in enquiry if (item['PURPOSE'] == 'Property Loan')]  # Total House Loan enquiries
            enq6ProL = [item for item in enquiry if
                        (item['DATEREPORTED_new'] > item['enquiries_L6m'] and item['PURPOSE'] == 'Property Loan')]
            enq12ProL = [item for item in enquiry if
                         (item['DATEREPORTED_new'] > item['enquiries_L12m'] and item['PURPOSE'] == 'Property Loan')]
            # Durable Loan Queries
            enq_bl = [item for item in enquiry if
                      (item['PURPOSE'] == 'Business Loan General')]  # Total Personal Loan enquiries
            enq6bl = [item for item in enquiry if
                      (item['DATEREPORTED_new'] > item['enquiries_L6m'] and item['PURPOSE'] == 'Business Loan General')]
            enq12bl = [item for item in enquiry if
                       (item['DATEREPORTED_new'] > item['enquiries_L12m'] and item['PURPOSE'] == 'Business Loan General')]
            # Auto-Loan Queries
            enq_al = [item for item in enquiry if (item['PURPOSE'] == 'Business Loan General')]  # Total Auto Loan enquiries
            enq6al = [item for item in enquiry if
                      (item['DATEREPORTED_new'] > item['enquiries_L6m'] and item['PURPOSE'] == 'Auto Loan (Personal)')]
            enq12al = [item for item in enquiry if
                       (item['DATEREPORTED_new'] > item['enquiries_L12m'] and item['PURPOSE'] == 'Auto Loan (Personal)')]
            # Two-Wheeler Loan Queries
            enq_cvl = [item for item in enquiry if
                       (item['PURPOSE'] == 'Commercial Vehicle Loan')]  # Total Auto Loan enquiries
            # Used car loan Queries
            enq_ucl = [item for item in enquiry if (item['PURPOSE'] == 'Used Car Loan')]  # Total Used Car Loan enquiries

            tot_enq, tot_enq_p10, tot_enq6, tot_enq12 = len(enquiry), len(enq_purp10), len(enq6m), len(enq12m)
            tot_pl, tot_enq6pl, tot_enq12pl = len(enq_loan), len(enq6pl), len(enq12pl)
            tot_ProL, tot_enq6ProL, tot_enq12ProL = len(enq_ProL), len(enq6ProL), len(enq12ProL)
            tot_bl, tot_enq6bl, tot_enq12bl = len(enq_bl), len(enq6bl), len(enq12bl)
            tot_al, tot_enq6al, tot_enq12al = len(enq_al), len(enq6al), len(enq12al)
            tot_cvl, tot_ucl = len(enq_cvl), len(enq_ucl)  # NEW FIX

            enquiry = [{**item, 'recent_enq_inMnths': round(item['days_diff_disb_To_enqDate'] / 30)} for item in enquiry]

            enquiry_rollup = {'tot_enq': tot_enq, 'CC_enq': tot_enq_p10, 'CC_enq_L6m': tot_enq6, 'CC_enq_L12m': tot_enq12,
                              'PL_enq': tot_pl, 'PL_enq_L6m': tot_enq6pl, 'PL_enq_L12m': tot_enq12pl,
                              'PROL_enq': tot_ProL, 'PROL_enq_L6m': tot_enq6ProL, 'PROL_enq_L12m': tot_enq12ProL,
                              'BUSSL_enq': tot_bl, 'BUSSL_enq_L6m': tot_enq6bl, 'BUSSL_enq_L12m': tot_enq12bl,
                              'AL_enq': tot_al, 'AL_enq_L6m': tot_enq6al, 'AL_enq_L12m': tot_enq12al,
                              'CVL_enq': tot_cvl, 'UsedCarL_enq': tot_ucl}

            enquiry_rollup['tot_AL_enq'] = enquiry_rollup['AL_enq'] + enquiry_rollup['CVL_enq'] + enquiry_rollup[
                'UsedCarL_enq']
            enquiry_rollup['other_enq'] = enquiry_rollup['tot_enq'] - enquiry_rollup['CC_enq'] - enquiry_rollup['PL_enq'] - \
                                          enquiry_rollup['PROL_enq'] - enquiry_rollup['BUSSL_enq'] - enquiry_rollup[
                                              'tot_AL_enq']

            min_recency = min(d['days_diff_disb_To_enqDate'] for d in enquiry)
            max_recency = max(d['days_diff_disb_To_enqDate'] for d in enquiry)
            enquiry = [{**item, 'min_recency': min_recency, 'max_recency': max_recency} for item in enquiry]
            enq_dedup = sorted(enquiry, key=lambda i: i['days_diff_disb_To_enqDate'])
            enq_dedup1, enq_dedup2 = enq_dedup[0], enq_dedup[-1]

            enquiry_rollup['most_recent_enq_amt'] = enq_dedup1['AMOUNT']
            enquiry_rollup['last_prod_enq'] = enq_dedup1['PURPOSE']
            enquiry_rollup['time_since_recent_enq'] = enq_dedup1['min_recency']
            enquiry_rollup['first_prod_enq'] = enq_dedup2[
                'PURPOSE']  # CHANGE: Earlier was EnquiryAmount, it should be PURPOSE

            enq_L1m = ['1' for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L1m'])]
            enq_L3m = ['1' for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L3m'])]
            enq_L6m = ['1' for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L6m'])]
            enq_L12m = ['1' for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L12m'])]
            Unsec_enq6m = [item for item in enquiry if (item['DATEREPORTED_new'] > item['enquiries_L6m'] and
                                                        item['PURPOSE'] in unsecured_list)]
            enquiry_rollup['enq_L1m'] = len(enq_L1m)
            enquiry_rollup['enq_L3m'] = len(enq_L3m)
            enquiry_rollup['enq_L6m'] = len(enq_L6m)
            enquiry_rollup['enq_L12m'] = len(enq_L12m)
            enquiry_rollup['Tot_Unsec_enq6m'] = len(Unsec_enq6m)
            enquiry_rollup['enq_L1M_by_enq_L3M'] = enquiry_rollup['enq_L1m'] / enquiry_rollup['enq_L3m'] if enquiry_rollup[
                                                                                                                'enq_L3m'] > 1 else None

            dateColData = [{'PROSPECTID': prospId, 'DATEREPORTED_new': item['DATEREPORTED_new'],
                            'DISBURSEDDATE': item['DISBURSEDDATE']} for item in enquiry]
            DATEREPORTED_new = [item['DATEREPORTED_new'] for item in enquiry]
            min_enq_rep_date = min(DATEREPORTED_new)
            max_enq_rep_date = max(DATEREPORTED_new)
            dateColData = dateColData[0]
            # enquiry_rollup['min_enq_rep_date'], enquiry_rollup['max_enq_rep_date'] = min_enq_rep_date, max_enq_rep_date
            enquiry_rollup['min_enq_rep_date'], enquiry_rollup['max_enq_rep_date'] = min_enq_rep_date.strftime(
                "%x"), max_enq_rep_date.strftime("%x")
            del DATEREPORTED_new, dateColData, enq_L3m, enq_L6m, enq_L12m;
            # if iddata is not None:
            #    iddata = [{**item, 'PAN_count': 1 if item['TYPE'] == '(PAN)' else 0, 'Aadhar_count': 1 if item['TYPE'] == '(UID)' else 0} for item in iddata]
            #    pan_sum, aadar_sum = sum(d['PAN_count'] for d in iddata), sum(d['Aadhar_count'] for d in iddata)
            #    enquiry_rollup['PAN_count'] = pan_sum
            #    enquiry_rollup['Aadhar_count'] = aadar_sum
            #    enquiry_rollup['Aadhar_PAN_Flag'] = 1 if (pan_sum == 1 and aadar_sum == 1) else 0
            # else:
            #    enquiry_rollup['PAN_count'] = None
            #    enquiry_rollup['Aadhar_count'] = None
            #    enquiry_rollup['Aadhar_PAN_Flag'] = None

            trades.update(enquiry_rollup)  # Merge trades and enquiry_rollup dictionaries.
            # TODO: Add this to case where only enquiry present
            if scoreList is not None:
                crifscore = scoreList['SCORE-VALUE']
                trades['CRIFSCORE_Treated'] = 0 if (crifscore in ['-', '000-1', 'NO RESPONSE', '1', '2', '3', '4',
                                                                  '5'] or crifscore == '' or crifscore == None) else crifscore
            else:
                trades['CRIFSCORE_Treated'] = 0

            account = get_tradeline(crif)
            account = [{**item, 'DISBDATE': reqDate} for item in account]

            account = [{**item, 'PaymentHistory': 'XXX' if 'COMBINED-PAYMENT-HISTORY' not in item or item[
                'COMBINED-PAYMENT-HISTORY'] == {} else str(
                format_payment(str(item['COMBINED-PAYMENT-HISTORY']))), 'DISBURSEDDATE': reqDate} for item in
                       account]
            min1 = min(len(item['PaymentHistory']) for item in account)
            max1 = max(len(item['PaymentHistory']) for item in account)
            if (min1 >= 3 | max1 <= 108):
                raise Exception('String length has an issue, either min is less than 3 or max is greater than 108')

            account = [{**item, 'DISBDATE': reqDate} for item in account]
            account = [{**d, 'PAYMENTHISTORY_ext_strngs': "Z" * (108 - len(d['PaymentHistory']))} for d in account]
            account = [{**d, 'PAYMENTHISTORY_full_strngs': d['PAYMENTHISTORY_ext_strngs'] + d['PaymentHistory'],
                        'PAYMENTHISTORYENDDATE': date_format_payment_crif(
                            d['CLOSED-DT'] if 'CLOSED-DT' in d and d['CLOSED-DT'] != {} else None),
                        'DATEOPENEDORDISBURSED': '' if 'DISBURSED-DT' not in d or d['DISBURSED-DT'] is None or d[
                            'DISBURSED-DT'] != {} else (d['DISBURSED-DT']),
                        'PAYMENTHISTORYSTARTDATE': date_format_payment_crif(
                            d['DISBURSED-DT'] if 'DISBURSED-DT' in d and d['DISBURSED-DT'] != {} else None),
                        'DATEREPORTEDANDCERTIFIED': date_format_payment_crif(
                            d['DATE-REPORTED'] if 'DATE-REPORTED' in d and d['DATE-REPORTED'] != {} else None)} for d in
                       account]

            min2 = min(len(item['PAYMENTHISTORY_full_strngs']) for item in account)
            max2 = max(len(item['PAYMENTHISTORY_full_strngs']) for item in account)
            if (min2 != max2):
                raise Exception('String length has should be same after filling empty strings')

            from dateutil import relativedelta
            account = [{**d, 'diff_years': relativedelta.relativedelta(d['DISBDATE'], d['PAYMENTHISTORYSTARTDATE'])} for d in account]
            account = [{**d, 'mon_diff_offset': abs(d['diff_years'].years) * 12 + d['diff_years'].months} for d in account]
            account = [{**d, 'offset_strings': "000" * d['mon_diff_offset']} for d in account]
            account = [{**d, 'PAYMENTGRID_with_offset': d['offset_strings'] + d['PAYMENTHISTORY_full_strngs']} for d in account]
        #     account = [{**d, 'PAYMENTGRID_with_offset_36m': d['PAYMENTGRID_with_offset'][:108]} for d in account]
            account = [{**d, 'PAYMENTGRID_with_offset_36m': d['PAYMENTGRID_with_offset'][len(d['PAYMENTGRID_with_offset']) - len(d['PAYMENTGRID_with_offset']):108]} for d in account]

            min3 = min(len(item['PAYMENTGRID_with_offset_36m']) for item in account)
            max3 = max(len(item['PAYMENTGRID_with_offset_36m']) for item in account)

            if (min3 != max3):
                raise Exception('Final Payment String length has should be same throughout')

            last_1m_date = reqDate - dateutil.relativedelta.relativedelta(months=1) + dateutil.relativedelta.relativedelta(
                months=1, days=-2)
            account = [{**d, 'time_sincerecent_prod': ((last_1m_date - date_format_payment_crif(
                d['DATEOPENEDORDISBURSED'])).days) / 30} for d in account]

            account = [{**d, 'HighCreditOrSanctionedAmount_New': 0 if 'DISBURSED-AMT' not in d else int(
                str(d['DISBURSED-AMT']).replace(',', '')),
                        'CurrentBalance_New': 0 if 'CURRENT-BAL' not in d or int(
                            str(d['CURRENT-BAL']).replace(',', '')) < 0 else int(str(d['CURRENT-BAL']).replace(',', '')),
                        'overdue_amount': 0 if 'OVERDUE-AMT' not in d else int(str(d['OVERDUE-AMT']).replace(',', ''))} for
                       d in account]
            sum_HighCreditOrSanctionedAmount = excl_none_operation(
                [d['HighCreditOrSanctionedAmount_New'] for d in account], sum)
            sum_current_bal = excl_none_operation([d['CurrentBalance_New'] for d in account], sum)
            invalid_values = [0, None]
            trades['high_credit_amount_by_bal'] = (
                        sum_HighCreditOrSanctionedAmount / sum_current_bal) if sum_current_bal not in invalid_values and sum_HighCreditOrSanctionedAmount is not None else None

            account = [{**d, 'time_sincerecent_prod': ((last_1m_date - date_format_payment_crif(
                d['DATEOPENEDORDISBURSED'])).days) / 30} for d in account]

            last_24m = reqDate - dateutil.relativedelta.relativedelta(months=24)
            x1 = [item for item in account if int(item['DATEREPORTEDANDCERTIFIED'] < last_24m)]

            # added on 21/10/2021   ---start
            trades['amt_overdue_total'] = excl_none_operation([d['overdue_amount'] for d in account], sum)
            last_12m = reqDate - dateutil.relativedelta.relativedelta(months=12)

            trades['amt_overdue_L24m'] = excl_none_operation([d['overdue_amount'] for d in x1], sum)

            # -----end 

            account = [{**d, 'PAYMENTGRID_with_offset_36mv1': split_grid(d['PAYMENTGRID_with_offset_36m'])} for
                       d in account]
            account = [{**d, 'PAYMENTGRID_with_offset_36mv2': list(reversed(d['PAYMENTGRID_with_offset_36mv1']))} for d in
                       account]
            account = [{**d, 'split_paygrid': (d['PAYMENTGRID_with_offset_36mv2'])} for d in account]



            account = [{**d, 'num_std': check_occurence(d['split_paygrid'], 'STD', None),
                        'num_std_6mts': check_occurence(d['split_paygrid'], 'STD', -6),
                        'num_std_12mts': check_occurence(d['split_paygrid'], 'STD', -12),
                        'num_sub': check_occurence(d['split_paygrid'], 'SUB', None),
                        'num_sub_6mts': check_occurence(d['split_paygrid'], 'SUB', -6),
                        'num_sub_12mts': check_occurence(d['split_paygrid'], 'SUB', -12),
                        'num_dbt': check_occurence(d['split_paygrid'], 'DBT', None),
                        'num_dbt_6mts': check_occurence(d['split_paygrid'], 'DBT', -6),
                        'num_dbt_12mts': check_occurence(d['split_paygrid'], 'DBT', -12),
                        'num_lss': check_occurence(d['split_paygrid'], 'LSS', None),
                        'num_lss_6mts': check_occurence(d['split_paygrid'], 'LSS', -6),
                        'num_lss_12mts': check_occurence(d['split_paygrid'], 'LSS', -12)} for d in account]

            account = [{**d, 'split_paygrid': [
                -1 if item in ['STD', 'XXX', 'ZZZ', 'OOO', 'DDD', 'SMA', 'LSS', 'SMA', 'SUB', '???', 'DBT', 'ZZ2', '.',
                               '60E', '+', 'ZZ0', '79E', '50O', 'np.nan', '06e', '26e', '08e', '00O', '0OO', 'OO0', 'O00',
                               'O', '0XX', '6XX', 'X00'] else int(item) for item in d['split_paygrid']]} for d in account]

            # added 21/10/2021 --start

            xx1 = [item for item in account if int(item['DATEREPORTEDANDCERTIFIED'] < last_12m)] 
            account = [{**d, 'spli_paygrid_12months':(d['split_paygrid']) } for d in xx1]
            # -- end
            account = [{**d, 'rvrs_split_paygrid': list(reversed(d['split_paygrid']))} for d in account]
            account = [{**d, 'recent_level_of_deliq': next(
                (j for i, j in enumerate(d['rvrs_split_paygrid']) if j > 0 and j is not None), None)} for d in account]

            account = [{**d,
                        'recnt_dpd_ind': next((i for i, j in enumerate(d['rvrs_split_paygrid']) if j > 0 and j is not None),
                                              None),
                        'old_dpd_ind': next((i for i, j in enumerate(d['split_paygrid']) if j > 0 and j is not None), None),
                        'filt_gt_0': list(filter(lambda x: (int(x) > 0), d['split_paygrid']))} for d in account]

            if len(account) > 0:
                first_trade = account[0]
                if first_trade['ACCT-TYPE'] == "Credit Card":
                    trades['First_Prod_CC_TL'] = 1
                else:
                    first_trade = {}
                    trades['First_Prod_CC_TL'] = 0
            # added 25/11/2021

            if len(account) > 0:
                first_trade = account[0]
                if first_trade['ACCT-TYPE'] in ['Credit Card','Non-Funded Credit Facility','Secured Credit Card','Corporate Credit Card','Kisan Credit Car']:
                    trades['First_Prod_R'] = 'Yes'
                else:
                    first_trade = {}
                    trades['First_Prod_R'] = 'No'



             # added 21/10/2021 .. num_times_30p_12m __start

            account = [{**d,'num_times_30p_12m':get_split_paygrid_res(d['spli_paygrid_12months'] ,30,None,None,'count',False)} for d in account]
            # ---- end
            account = [{**d, 'maxdpdval': max(d['filt_gt_0']) if len(d['filt_gt_0']) > 0 else None,
                        'mindpdval': min(d['filt_gt_0']) if len(d['filt_gt_0']) > 0 else None,
                        'time_since_recnt_deliq': (d['recnt_dpd_ind'] + 1) if d['recnt_dpd_ind'] is not None else -99999,
                        'time_since_oldest_deliq': 36 - d['old_dpd_ind'] - 1 if d['old_dpd_ind'] is not None else None,
                        'num_times_deliq': get_split_paygrid_res(d['split_paygrid'], 0, None, None, 'count',
                                                                              False),
                        # Dec 23 - Change sum to count
                        'num_times_30p': get_split_paygrid_res(d['split_paygrid'], 30, None, None, 'count',
                                                                            False),
                        'num_times_60p': get_split_paygrid_res(d['split_paygrid'], 60, None, None, 'count',
                                                                            False),
                        'num_times_90p': get_split_paygrid_res(d['split_paygrid'], 90, None, None, 'count',
                                                                            False),
                        'num_deliq_6mts': get_split_paygrid_res(d['split_paygrid'], 0, -6, None, 'count',
                                                                             False),
                        'num_deliq_12mts': get_split_paygrid_res(d['split_paygrid'], 0, -12, None, 'count',
                                                                              False),
                        'num_deliq_6_12mts': get_split_paygrid_res(d['split_paygrid'], 0, -12, -6, 'count',
                                                                                False),
                        'max_deliq_6mts': get_split_paygrid_res(d['split_paygrid'], 0, -6, None, 'max', True),
                        'max_deliq_12mts': get_split_paygrid_res(d['split_paygrid'], 0, -12, None, 'max',
                                                                              True),
                        'max_deliq_6_12mts': get_split_paygrid_res(d['split_paygrid'], 0, -12, -6, 'max', True)
                        } for d in account]

            new_acc = [{'time_since_first_deliquency': d['time_since_oldest_deliq'],
                        'time_since_recent_deliquency': d['time_since_recnt_deliq'],
                        'num_times_delinquent': d['num_times_deliq'], 'max_delinquency_level': d['maxdpdval'],
                        'num_deliq_6mts': d['num_deliq_6mts'], 'num_deliq_12mts': d['num_deliq_12mts'],
                        'num_deliq_6_12mts': d['num_deliq_6_12mts'], 'max_deliq_6mts': d['max_deliq_6mts'],
                        'max_deliq_12mts': d['max_deliq_12mts'], 'max_deliq_6_12mts': d['max_deliq_6_12mts'],
                        'num_times_30p_dpd': d['num_times_30p'],'num_times_30p_dpd_12m':d['num_times_30p_12m'], 'num_times_60p_dpd': d['num_times_60p'],
                        'num_times_90p_dpd': d['num_times_90p'],
                        'num_std': d['num_std'], 'num_std_6mts': d['num_std_6mts'],
                        'num_std_12mts': d['num_std_12mts'], 'num_sub': d['num_sub'],
                        'num_sub_6mts': d['num_sub_6mts'], 'num_sub_12mts': d['num_sub_12mts'],
                        'num_dbt': d['num_dbt'], 'num_dbt_6mts': d['num_dbt_6mts'],
                        'num_dbt_12mts': d['num_dbt_12mts'], 'num_lss': d['num_lss'],
                        'num_lss_6mts': d['num_lss_6mts'], 'num_lss_12mts': d['num_lss_12mts'],
                        'recent_level_of_deliq': d['recent_level_of_deliq'],
                        'max_recent_level_of_deliq': d['recent_level_of_deliq'],
                        'num_tradelines': 1} for d in account]

            x1 = [item for item in account if (item['num_times_60p'] > 1, 0)]
            trades['num_TL_60DPD_L12M'] = sum(d['num_times_60p'] for d in x1)

            sorted_acc = sorted(new_acc, key=lambda i: (i['time_since_recent_deliquency'], i['recent_level_of_deliq']))
            if len(sorted_acc) > 0:
                sorted_acc = sorted_acc[0]
                sorted_acc['time_since_recent_deliquency'] = None if sorted_acc[
                                                                         'time_since_recent_deliquency'] == -99999 else \
                sorted_acc['time_since_recent_deliquency']
            else:
                sorted_acc = {}
                sorted_acc['time_since_recent_deliquency'] = None

            new_acc = [{**d, 'num_TL_60DPDinL12M': 1 if d['num_times_60p_dpd'] > 0 or d['num_times_60p_dpd'] > 1 or d[
                'num_times_60p_dpd'] > 0 else 0} for d in new_acc]
            new_acc = [{**d, 'tot_deliq_TLs': 1 if d['num_times_delinquent'] > 0 or d['num_times_delinquent'] > 1 or d[
                'num_times_delinquent'] > 0 else 0} for d in new_acc]

            trades['time_since_first_deliquency'] = excl_none_operation(
                [d['time_since_first_deliquency'] for d in new_acc], 'sum')
            trades['time_since_recent_deliquency'] = excl_none_operation(
                [d['time_since_recent_deliquency'] for d in new_acc], 'min')
            trades['num_times_delinquent'] = excl_none_operation([d['num_times_delinquent'] for d in new_acc],
                                                                              'sum')
            trades['max_delinquency_level'] = excl_none_operation(
                [d['max_delinquency_level'] for d in new_acc], 'max')
            trades['max_recent_level_of_deliq'] = excl_none_operation(
                [d['max_recent_level_of_deliq'] for d in new_acc], 'max')
            # added 25/11/2021 --recent deliq
            trades['recent_deliq']=new_acc[0]['recent_level_of_deliq'] if (len(new_acc)>0 and new_acc[0]['recent_level_of_deliq']) else -99999
            trades['num_deliq_6mts'] = excl_none_operation([d['num_deliq_6mts'] for d in new_acc], 'sum')
            trades['num_deliq_12mts'] = excl_none_operation([d['num_deliq_12mts'] for d in new_acc], 'sum')
            trades['num_deliq_6_12mts'] = excl_none_operation([d['num_deliq_6_12mts'] for d in new_acc], 'sum')
            trades['num_tradelines'] = excl_none_operation([d['num_tradelines'] for d in new_acc], 'sum')
            trades['max_deliq_6mts'] = excl_none_operation([d['max_deliq_6mts'] for d in new_acc], 'max')
            trades['max_deliq_12mts'] = excl_none_operation([d['max_deliq_12mts'] for d in new_acc], 'max')
            trades['max_deliq_6_12mts'] = excl_none_operation([d['max_deliq_6_12mts'] for d in new_acc], 'max')
            trades['num_times_30p_dpd'] = excl_none_operation([d['num_times_30p_dpd'] for d in new_acc], 'sum')
            trades['num_times_30p_dpd_12m'] = excl_none_operation([d['num_times_30p_dpd_12m'] for d in new_acc], 'sum')
            trades['num_60dpd_TL'] = sum(1 for d in new_acc if d['num_times_60p_dpd'] > 0)
            trades['num_times_60p_dpd'] = excl_none_operation([d['num_times_60p_dpd'] for d in new_acc], 'sum')
            trades['num_times_90p_dpd'] = excl_none_operation([d['num_times_90p_dpd'] for d in new_acc], 'sum')
            trades['num_std'] = excl_none_operation([d['num_std'] for d in new_acc], 'sum')
            trades['num_std_6mts'] = excl_none_operation([d['num_std_6mts'] for d in new_acc], 'sum')
            trades['num_std_12mts'] = excl_none_operation([d['num_std_12mts'] for d in new_acc], 'sum')
            trades['num_sub'] = excl_none_operation([d['num_sub'] for d in new_acc], 'sum')
            trades['num_sub_6mts'] = excl_none_operation([d['num_sub_6mts'] for d in new_acc], 'sum')
            trades['num_sub_12mts'] = excl_none_operation([d['num_sub_12mts'] for d in new_acc], 'sum')
            trades['num_dbt'] = excl_none_operation([d['num_dbt'] for d in new_acc], 'sum')
            trades['num_dbt_6mts'] = excl_none_operation([d['num_dbt_6mts'] for d in new_acc], 'sum')
            trades['num_dbt_12mts'] = excl_none_operation([d['num_dbt_12mts'] for d in new_acc], 'sum')
            trades['num_lss'] = excl_none_operation([d['num_lss'] for d in new_acc], 'sum')
            trades['num_lss_6mts'] = excl_none_operation([d['num_lss_6mts'] for d in new_acc], 'sum')
            trades['num_lss_12mts'] = excl_none_operation([d['num_lss_12mts'] for d in new_acc], 'sum')
            trades['time_since_recent_prod'] = excl_none_operation(
                [d['time_sincerecent_prod'] for d in account], 'min')
            trades['num_TL_60DPD_L12M'] = excl_none_operation([d['num_TL_60DPDinL12M'] for d in new_acc],
                                                                           'sum')
            trades['tot_deliq_TLs'] = excl_none_operation([d['tot_deliq_TLs'] for d in new_acc], 'sum')
            # added 25/11/2021
            trades['CRIF_Bureau_score'] = trades['CRIFSCORE_Treated']
            trades['First_Prod_R'] = trades['First_Prod_R']

            dev_data = trades
            invalid_values = [0, None]
            # Dec 23 - Change - convert to float for calculating percentage variables
            dev_data['pct_of_active_TLs_ever'] = round(float(dev_data['Tot_Active_TL']) / float(dev_data['Total_TL']), 3) if \
            dev_data['Total_TL'] not in invalid_values and dev_data['Tot_Active_TL'] is not None else None
            dev_data['pct_opened_TLs_L6m_of_L12m'] = round(
                float(dev_data['Total_TL_opened_L6M']) / float(dev_data['Total_TL_opened_L12M']), 3) if dev_data[
                                                                                                            'Total_TL_opened_L12M'] not in invalid_values and \
                                                                                                        dev_data[
                                                                                                            'Total_TL_opened_L6M'] is not None else None
            # dev_data['pct_currentBal_all_TL'] = round(float(dev_data['Tot_Currentbalance']) / float(dev_data['Tot_Sactioned_amt']), 3) if dev_data['Tot_Sactioned_amt'] not in invalid_values and dev_data['Tot_Currentbalance'] is not None else None
            dev_data['CC_Flag'] = '1' if dev_data['CC_TL'] > 0 else '0'
            dev_data['PL_Flag'] = '1' if dev_data['PL_TL'] > 0 else '0'

            dev_data['pct_Of_securedLoan'] = round(float(dev_data['Secured_TL']) / float(dev_data['Total_TL']), 3) if \
            dev_data['Total_TL'] not in invalid_values and dev_data['Secured_TL'] is not None else None
            dev_data['pct_Of_securedLoan'] = 0.0 if dev_data['pct_Of_securedLoan'] is None else dev_data[
                'pct_Of_securedLoan']
            dev_data['pct_Of_securedLoan'] = 0.0 if dev_data['pct_Of_securedLoan'] < 0 else dev_data['pct_Of_securedLoan']
            dev_data['pct_Of_UnSecuredLoan'] = round(1 - dev_data['pct_Of_securedLoan'], 3)
            dev_data['pct_tot_deliq_TLs'] = round(float(dev_data['num_times_delinquent']) / float(dev_data['Total_TL']),
                                                  3) if dev_data['Total_TL'] not in invalid_values and dev_data[
                'num_times_delinquent'] is not None else None  # if dev_data['num_times_delinquent'] is not None and dev_data['Total_TL'] is not None else 99999
            dev_data['pct_Satis_TL_to_Tot_TL'] = round(
                float(dev_data['num_tradelines'] - dev_data['tot_deliq_TLs']) / float(dev_data['num_tradelines']) * 100,
                3) if dev_data['num_tradelines'] not in invalid_values and dev_data['num_tradelines'] is not None else None

            dev_data['Auto_TL_cat'] = '99999' if dev_data['Auto_TL'] > 4 else str(dev_data['Auto_TL'])
            dev_data['time_since_recent_payment'] = dev_data['Time_Since_Payment']
            dev_data['TIME_SINCE_RECENT_PAYMENT_INMNTHS'] = dev_data['Time_Since_Last_Payment_inMnths']
            dev_data['HL_Flag'] = 1 if dev_data['Home_TL'] > 0 else 0
            dev_data['GL_Flag'] = 1 if dev_data['Gold_TL'] > 0 else 0
            dev_data['max_deliq_6mts'] = 0 if dev_data['max_deliq_6mts'] is None else dev_data['max_deliq_6mts']
            dev_data['max_deliq_12mts'] = 0 if dev_data['max_deliq_12mts'] is None else dev_data['max_deliq_12mts']
            dev_data['Delq_Inc_L6M_L12M'] = 1 if dev_data['max_deliq_6mts'] > dev_data['max_deliq_12mts'] else 0
            dev_data['last_prod_enq2'] = last_first_prod_enq(dev_data['last_prod_enq'])
            dev_data['first_prod_enq2'] = last_first_prod_enq(dev_data['first_prod_enq'])
            dev_data['num_times_delinquent'] = 0 if dev_data['num_times_delinquent'] is None else dev_data[
                'num_times_delinquent']
            dev_data['delinquent_flag'] = 'Yes' if dev_data['num_times_delinquent'] > 0 else 'No'
            dev_data['crif_status']='Yes'
            dev_data['error_reason'] = ''
            new_rollup_var = dict()

            # added unsecured tl and recent deliq 25/11/2021
            for k, v in dev_data.items():
                if k in {'error_reason','Age_Oldest_TL', 'enq_L6m', 'enq_L12m', 'num_60dpd_TL', 'num_times_90p_dpd', 'pct_Satis_TL_to_Tot_TL', 'time_since_recent_deliquency', 'util', 'amt_overdue_L24m', 'Num_Open_CC_TL', 'enq_L1M_by_enq_L3M', 'max_recent_level_of_deliq', 'Num_Closed_Unsecured_TL', 'Tot_Revolving_Bal', 'Tot_Closed_TL', 'tot_deliq_TLs', 'Total_TL', 'First_Prod_CC_TL', 'Avg_Age_Act', 'Total_cur_blnc_to_Original_amount','enq_L3m',
        'Total_TL_opened_L3M','WrittenoffAmount_L36M','First_Prod_R','CRIF_Bureau_score','WrittenoffAmount_L12M','recent_deliq','amt_overdue_total','num_times_30p_dpd_12m','debt_curr_bal','Unsecured_TL','crif_status'}:
                    new_rollup_var[k] = v
            print(len(list(new_rollup_var)))

        # added else condition on 23/12/2021
        else:
            new_rollup_var ={
                'error_reason':'No data',
                'Age_Oldest_TL':-99999,
                'enq_L6m':-99999,
                'enq_L12m':-99999,
                'num_60dpd_TL':-99999,
                'num_times_90p_dpd':-99999,
                'pct_Satis_TL_to_Tot_TL':-99999,
                'time_since_recent_deliquency':-99999,
                'util':-99999,
                'amt_overdue_L24m':-99999,
                'Num_Open_CC_TL':-99999,
                'enq_L1M_by_enq_L3M':-99999,
                'max_recent_level_of_deliq':-99999,
                'Num_Closed_Unsecured_TL':-99999,
                'Tot_Revolving_Bal':-99999,
                'Tot_Closed_TL':-99999,
                'tot_deliq_TLs':-99999,
                'Total_TL':-99999, 
                'First_Prod_CC_TL':-99999,
                'Avg_Age_Act':-99999,
                'Total_cur_blnc_to_Original_amount':-99999,
                'enq_L3m':-99999,
                'Total_TL_opened_L3M':-99999,
                'WrittenoffAmount_L36M':-99999,
                'First_Prod_R':-99999,
                'CRIF_Bureau_score':-99999,
                'WrittenoffAmount_L12M':-99999,
                'recent_deliq':-99999,
                'amt_overdue_total':-99999,
                'num_times_30p_dpd_12m':-99999,
                'debt_curr_bal':-99999,
                'Unsecured_TL':-99999,
                'crif_status':'No'
            }
    except Exception  as e:
        new_rollup_var ={
                'error_reason':'CRIF pull failed',
                'Age_Oldest_TL':-99999,
                'enq_L6m':-99999,
                'enq_L12m':-99999,
                'num_60dpd_TL':-99999,
                'num_times_90p_dpd':-99999,
                'pct_Satis_TL_to_Tot_TL':-99999,
                'time_since_recent_deliquency':-99999,
                'util':-99999,
                'amt_overdue_L24m':-99999,
                'Num_Open_CC_TL':-99999,
                'enq_L1M_by_enq_L3M':-99999,
                'max_recent_level_of_deliq':-99999,
                'Num_Closed_Unsecured_TL':-99999,
                'Tot_Revolving_Bal':-99999,
                'Tot_Closed_TL':-99999,
                'tot_deliq_TLs':-99999,
                'Total_TL':-99999, 
                'First_Prod_CC_TL':-99999,
                'Avg_Age_Act':-99999,
                'Total_cur_blnc_to_Original_amount':-99999,
                'enq_L3m':-99999,
                'Total_TL_opened_L3M':-99999,
                'WrittenoffAmount_L36M':-99999,
                'First_Prod_R':-99999,
                'CRIF_Bureau_score':-99999,
                'WrittenoffAmount_L12M':-99999,
                'recent_deliq':-99999,
                'amt_overdue_total':-99999,
                'num_times_30p_dpd_12m':-99999,
                'debt_curr_bal':-99999,
                'Unsecured_TL':-99999,
                'crif_status':'No'
            }
        
        
    return new_rollup_var

<>:54: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:54: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-99-cd7a4cc9fa97>:54: SyntaxWarning: "is" with a literal. Did you mean "=="?
  crif_tradlines = [{**item, 'MissInd_Pmnt_date': 1 if item['CurrentBalance'] > 0 is None and item[


In [100]:
rollup_crif(prospId, crif, reqDate)

[]
33


{'Total_TL': 9,
 'Tot_Closed_TL': 5,
 'Avg_Age_Act': 33.114814814814814,
 'Total_TL_opened_L3M': 0,
 'Unsecured_TL': 0,
 'Num_Open_CC_TL': 0,
 'Num_Closed_Unsecured_TL': 0,
 'Age_Oldest_TL': 99.46666666666667,
 'debt_curr_bal': 125610,
 'WrittenoffAmount_L12M': 0,
 'WrittenoffAmount_L36M': 0,
 'Tot_Revolving_Bal': None,
 'Total_cur_blnc_to_Original_amount': 2.149438738953905,
 'util': -99999,
 'enq_L3m': 0,
 'enq_L6m': 0,
 'enq_L12m': 26,
 'enq_L1M_by_enq_L3M': None,
 'amt_overdue_total': 446.6666666666667,
 'amt_overdue_L24m': 446.6666666666667,
 'First_Prod_CC_TL': 0,
 'First_Prod_R': 'No',
 'time_since_recent_deliquency': -99999,
 'max_recent_level_of_deliq': None,
 'recent_deliq': -99999,
 'num_times_30p_dpd_12m': 0,
 'num_60dpd_TL': 0,
 'num_times_90p_dpd': 0,
 'tot_deliq_TLs': 0,
 'CRIF_Bureau_score': 684,
 'pct_Satis_TL_to_Tot_TL': 100.0,
 'crif_status': 'Yes',
 'error': ''}